In [7]:
from rdflib import Graph, Namespace, RDF
import pandas as pd

# 1. Load your schema and instance files
g = Graph()
g.parse("tourisme.ttl", format="turtle")
g.parse("tourisme_ins.ttl", format="turtle")

# 2. Define prefixes for SPARQL
TS = Namespace("http://example.org/tourisme#")

PREFIXES = """
PREFIX :    <http://example.org/tourisme#>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
"""

In [8]:
# Helper to run a query and return a DataFrame
def sparql_df(query, cols, caster):
    res = g.query(PREFIXES + query)
    rows = [caster(r) for r in res]
    return pd.DataFrame(rows, columns=cols)

In [11]:
# 3. Query 1: Toutes les activités
q1 = """
SELECT ?activite ?nom ?ville ?capacity WHERE {
  ?activite rdf:type :Activite ;
             :hasName        ?nom ;
             :locatedIn      ?ville ;
             :capacity       ?capacity .
}
"""
df1 = sparql_df(
    q1,
    ["Activite","Nom","Ville","Capacite"],
    lambda r: [r.activite.split("#")[-1], str(r.nom), r.ville.split("#")[-1], int(r.capacity)]
)
df1

,Activite,Nom,Ville,Capacite
0,KayakSeine,Kayak sur la Seine,Paris,12
1,LouvreVisit,Visite guidée du Louvre,Paris,20
2,WineTour,Dégustation de vins en Provence,Provence,15


In [12]:
# 4. Query 2: Hébergements
q2 = """
SELECT ?heberg ?nom ?ville ?prix WHERE {
  ?heberg rdf:type/rdfs:subClassOf* :Hebergement ;
          :hasName    ?nom ;
          :hasPrice   ?prix ;
          :locatedIn  ?ville .
}
"""
df2 = sparql_df(
    q2,
    ["Hebergement","Nom","Ville","Prix"],
    lambda r: [r.heberg.split("#")[-1], str(r.nom), r.ville.split("#")[-1], float(r.prix)]
)
df2

,Hebergement,Nom,Ville,Prix
0,HotelLouvre,Hôtel du Louvre,Paris,150.0
1,MaisonDuSud,Maison du Sud,Provence,80.0


In [13]:
# 5. Query 3: Touristes et leurs activités
q3 = """
SELECT ?touriste ?nom ?email ?heberge
       (GROUP_CONCAT(STRAFTER(STR(?act), "#"); separator=", ") AS ?activites)
WHERE {
  ?touriste rdf:type :Touriste ;
            foaf:name               ?nom ;
            :email                  ?email ;
            :hasAccommodation       ?heberge ;
            :participatesInActivity ?act .
}
GROUP BY ?touriste ?nom ?email ?heberge
"""
df3 = sparql_df(
    q3,
    ["Touriste","Nom","Email","Hebergement","Activites"],
    lambda r: [
        r.touriste.split("#")[-1],
        str(r.nom),
        str(r.email),
        r.heberge.split("#")[-1],
        str(r.activites)
    ]
)
df3

,Touriste,Nom,Email,Hebergement,Activites
0,Ali,Ali Mellah,ali@example.com,HotelLouvre,"KayakSeine, LouvreVisit"
1,Marie,Marie Curie,marie@example.com,MaisonDuSud,WineTour


In [14]:
# 6. Query 4: Guides et tours qu’ils gèrent
q4 = """
SELECT ?guide ?nom ?tour WHERE {
  ?guide rdf:type :GuideTouristique ;
         foaf:name        ?nom ;
         :managesTour     ?tour .
}
"""
df4 = sparql_df(
    q4,
    ["Guide","Nom","Tour"],
    lambda r: [r.guide.split("#")[-1], str(r.nom), r.tour.split("#")[-1]]
)
df4

,Guide,Nom,Tour
0,PierreGuide,Pierre Dupont,LouvreVisit
